In [1]:
# GET PARAMETERS
import os 
import sys
import itertools
import numpy as np 
from bokeh.plotting import output_notebook,show ,column
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_and_visu_non_recurrent import get_ds
from examples.train_model_on_k_fold_validation import load_configuration
from plotting.plot_distribution_inputs import plot_scatter_distribution_by_calendar_group,plot_distribution_by_calendar_group,get_usefull_params

Training and Hyper-parameter tuning with Ray is not possible


In [5]:
trial_id ='subway_in_STGCN_MSELoss_2025_01_06_08_00_94523'

args,_ = load_configuration(trial_id,load_config=True)


station = ['BEL','PER','PAR','GER','CHA']   # 'BON'  #'GER'
training_mode_to_visualise = ['test']#,'valid','train']
modification ={'keep_best_weights':True,
                'epochs':2,
                'validation_split_method' : 'forward_chaining_cv',
                'min_fold_size_proportion': 0.75,
                'train_prop':0.6,
                'valid_prop':0.2,
                'test_prop':0.2,
                'freq':'15min',

                #'set_spatial_units':station,
        
                'data_augmentation': False, #True,  #False
                }

# Preprocesss / Init
def preprocess_calendar_possibilities(ds,training_mode):
    t_minus_1 = getattr(ds.tensor_limits_keeper,f"df_verif_{training_mode}").iloc[:,-2].reset_index(drop=True)
    s_hours = t_minus_1.dt.hour
    s_weekdays = t_minus_1.dt.weekday
    s_minutes = t_minus_1.dt.minute

    posible_hours = s_hours.unique()
    posible_hours.sort()
    posible_weekdays = s_weekdays.unique()
    posible_weekdays.sort()
    posible_minutes = s_minutes.unique()
    posible_minutes.sort()
    return t_minus_1,s_hours,s_weekdays,s_minutes,posible_hours,posible_weekdays,posible_minutes


>>>> Load best CONFIG


In [ ]:
ds,args_modif,trial_id,save_folder,df_loss = get_ds(args.model_name,args.dataset_names,args.dataset_for_coverage,
                                              modification=modification,args_init=args,fold_to_evaluate=[args.K_fold-1])

## Plot Distributino of the Training Set 
- Le training set peut contenir de la Data Augmentation si le Trial considéré en à utilisé un. 
    - 'subway_in_STGCN_MSELoss_2025_01_06_08_00_94523' à par exemple les mardi et jeudi 23h-00h augmenté par interpolation (x2)
- 'agg' choisi le type d'aggreagtion temporelle. Si on aggrègre pas du tout, ou si on aggrègre par heure.
- station_ind parcours les stations qui nous intéresse. Il y a 40 stations, donc théoriquement on pourrait faire : 
    - for station_ind in np.arange(len(ds.spatial_unit)): 

In [9]:
agg = None # 'weekday_hour'  # Choices: [None,'hour','weekday_hour']

for agg in [None,'hour']:
    for training_mode in ['train','test']:
        print('training_mode: ',training_mode)
        t_minus_1,s_hours,s_weekdays,s_minutes,posible_hours,posible_weekdays,posible_minutes = preprocess_calendar_possibilities(ds,training_mode)
        for station_ind in [1,25]:#np.arange(len(ds.spatial_unit)): #[0]: # np.arange(len(ds.spatial_unit)):
            # Load data to plot
            add_to_point_to_plots,tuples,x_ticks = get_usefull_params(ds,t_minus_1,agg,posible_weekdays,posible_hours,posible_minutes,s_weekdays,s_hours,s_minutes,training_mode)
            list_of_point_to_plots = add_to_point_to_plots(tuples,station_ind)

            # plot 
            box_fig = plot_distribution_by_calendar_group(list_of_point_to_plots,x_ticks,ds.spatial_unit[station_ind])
            scatter_fig = plot_scatter_distribution_by_calendar_group(list_of_point_to_plots,x_ticks,ds.spatial_unit[station_ind],height=300, width=1300)
            # Afficher
            #show(column(scatter_fig, box_fig))
            #show(box_fig)
            output_notebook()
            show(scatter_fig)

training_mode:  train


Loading BokehJS ...

Loading BokehJS ...

training_mode:  test


Loading BokehJS ...

...

Loading BokehJS ...

Loading BokehJS ...

training_mode:  test


Loading BokehJS ...

Loading BokehJS ...